In [2]:
import numpy as np
import matplotlib.pyplot as plt
#Import qutip
from qutip import *

#### 1. Implement Gate Fidelity

In [11]:
def state_fidelity_after_gate(rho_in, ideal_gate, noisy_gate):

    rho_ideal = ideal_gate * rho_in * ideal_gate.dag()

    rho_noisy = noisy_gate * rho_in * noisy_gate.dag()

    return fidelity(rho_noisy, rho_ideal)

def rho_TR(d):

    psi = sum(basis(d, i) for i in range(d)).unit()
    return ket2dm(psi)

def apply_unitary(U, rho):

    return U * rho * U.dag()

def F_TR(d, ideal_gate, noisy_gate):

    rho_in = rho_TR(d)

    rho_ideal = apply_unitary(ideal_gate, rho_in)

    rho_real = apply_unitary(noisy_gate, rho_in)

    return fidelity(rho_real, rho_ideal)  


def F_arith(d, ideal_gate, noisy_gate):

    F_basis_sum = 0.0

    for i in range(d):

        ket_i = basis(d,i)
        rho_i = ket2dm(ket_i)
        F_basis_sum += state_fidelity_after_gate(rho_i, ideal_gate, noisy_gate)
    

    return (F_basis_sum + F_TR(d, ideal_gate, noisy_gate)) / (d + 1)


def F_geom(d, ideal_gate, noisy_gate):
    
     F_basis_product = 1.0

     for i in range(d):
         
         ket_i = basis(d,i)
         rho_i = ket2dm(ket_i)
         F_basis_product *= state_fidelity_after_gate(rho_i, ideal_gate, noisy_gate)
         
     return (1 / (d + 1)) + ((1 - (1 / (d + 1))) * (F_TR(d, ideal_gate, noisy_gate) * F_basis_product))

def F_lambda(d, ideal_gate, noisy_gate):

    F_basis_product = 1.0

    for i in range(d):
         
        ket_i = basis(d,i)
        rho_i = ket2dm(ket_i)
        F_basis_product *= state_fidelity_after_gate(rho_i, ideal_gate, noisy_gate)

    lambd = 1 - ((1 - F_basis_product) / (1 - (F_basis_product * F_TR(d, ideal_gate, noisy_gate))))

    return (lambd * F_geom(d, ideal_gate, noisy_gate)) + ((1 - lambd) * F_arith(d, ideal_gate, noisy_gate))


In [31]:
d = 4
U = rand_unitary(d)
print(abs(F_lambda(d, U, U) - 1.0))

1.8373060850507272e-08
